# Scenario Data Post-Processing from Model Output

## Description
The script extracts from the output tables the overall costs, emissions and fuel amounts per fuel type and its sum.

Scenario name and rate have to be provided first.

## Content
>- Script set-up
- Scenario and rate
- Load data
- Required directories
- Data preparation
- Store data
- Combine data
- Store final data frame

## Script Set-Up

In [98]:
# Import of required packages
import os
import pandas as pd
import glob

## Scenario and Rate

In [99]:
# Reference scenario: rs
# Regulation scenarios: bau, rs_mp, snox, imo, tdvar
# Cost scenarios: batwind, bdo, ch3oh, lbg, lng, lbg_mp, lng_mp, h2, nh3

r = 0.2
R = str(int(r*100))
scn = 'h2'

## Load Data

In [100]:
# Load data from model output directory
path = 'model_output/' + R

df = pd.read_csv(path + '/results_%s.csv' % scn, encoding='utf8', index_col=[0, 1])

## Required Directories

In [101]:
# Make required directories
path = 'tables/'

if not os.path.isdir(path):
    os.makedirs(path)

## Data Preparation

In [102]:
# Reset index
df = df.reset_index()

# Make new column for each ship types fuel
df['fuel'] = df['Unnamed: 1'].str.split('_').str[1]

# Group per fuel and sum over all years
df = df.groupby(['fuel'])[['CF', 'CI', 'CS', 'EC', 'EM', 'fa']].sum()

In [103]:
# Make the sum for each row over all fuels
df.loc['total'] = df.sum()

# Unstack the data frame
df = df.unstack()

# Make a data frame
df = pd.DataFrame(df)

# Rename column name
df.columns = [scn + '_r' + R]

## Store Data

In [104]:
# Write data to csv
file = 'results_' + scn + '_r' + R + '_processed.csv'
df.to_csv(path + file, encoding='utf8')

## Combine Data

In [105]:
# Load all result files from the tables directory
all_files = glob.glob(path + 'results' + '*.csv')

In [106]:
all_files

['tables\\results_bau_r0_processed.csv',
 'tables\\results_rs_mp_r0_processed.csv',
 'tables\\results_snox_r0_processed.csv',
 'tables\\results_imo_r0_processed.csv',
 'tables\\results_tdvar_r0_processed.csv',
 'tables\\results_h2_r20_processed.csv',
 'tables\\results_nh3_r-20_processed.csv',
 'tables\\results_lbg_r-40_processed.csv',
 'tables\\results_lbg_mp_r-40_processed.csv',
 'tables\\results_ch3oh_r-20_processed.csv',
 'tables\\results_lng_r-80_processed.csv',
 'tables\\results_bdo_r-50_processed.csv',
 'tables\\results_lng_mp_r-80_processed.csv',
 'tables\\results_batwind_r-99_processed.csv',
 'tables\\results_rs_r0_processed.csv']

In [112]:
# Concat all files into one data frame
df1 = pd.DataFrame()
for f in all_files:
    df2 = pd.read_csv(f, encoding='utf8', index_col=[0, 1])
    new_col = df2.columns[0]
    df1[new_col] = df2[new_col]

# Round to 2 decimals
df1 = df1.round(2)

# Drop empty rows
df_final = df1[(df1.T >= 0.01).any()]

In [113]:
df_final.loc[('co2e_price', 'total'), :] = 0
for col in df_final.columns:
    df_final.loc[('co2e_price', 'total'), col] = ((df_final.loc[('CF', 'total'), col] +
                                                  df_final.loc[('CI', 'total'), col] +
                                                  df_final.loc[('CS', 'total'), col] -
                                                  df_final.loc[('CF', 'total'), 'bau_r0'] -
                                                  df_final.loc[('CI', 'total'), 'bau_r0'] -
                                                  df_final.loc[('CS', 'total'), 'bau_r0']) /
                                                  (df_final.loc[('EC', 'total'), 'bau_r0'] +
                                                  df_final.loc[('EM', 'total'), 'bau_r0'] -
                                                  df_final.loc[('EC', 'total'), col] - 
                                                  df_final.loc[('EM', 'total'), col])) * 10**6
df_final = df_final.round(2)

C:\Users\tilseb\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [114]:
# Convert costs from 10^12 to 10^9
df_final.loc[['CF', 'CI', 'CS'], :] *= 1000

## Store Final Data Frame

In [115]:
# Write data to csv
file = 'final_results_processed.csv'
df_final.to_csv(path + file, encoding='utf8')